In [182]:
import json
import re

import requests
import scrapy

In [183]:
base_url = 'http://goheels.com'
url = base_url + '/roster.aspx?path=mbball'
resp = requests.get(url, headers={'User-Agent': 'UNC Journo Class'},)

In [184]:
body_bytes = resp.content

body_str = body_bytes.decode('utf-8')

body_str

'\r\n\r\n<!doctype html>\r\n<html id="ctl00_html" lang="en" class=" wide mbball sport mbball"> \r\n    <head><script>window.client_hostname = "goheels.com";window.server_name = "866413-LB18";window.client_division = "DI";window.targetting_criteria = {"page_template":"roster-list","sport_name":"mbball","sport_name_custom":"mbball","site":"unc","content_id":null,"sect":"mbaskbl"};window.client_shortname = "unc";window.dfp_network_code = "29658103";window.img_dfp_unit_name = "";window.block_dfp = false;window.reactions = null;</script><link rel="stylesheet" href="/less/sidearm-responsive-grid.less?_=636518877046678376"><link rel="stylesheet" href="https://s3.amazonaws.com/assets.sidearmsports.com/adaptive/css/normalize.min.css"><link rel="apple-touch-icon" href="/images/logos/site/site.png"><link rel="canonical" href="http://goheels.com/roster.aspx?roster=476&amp;path=mbball"><meta charset="utf-8"><meta name="viewport" content="width=device-width"><link rel="apple-touch-icon" href="/image

In [185]:
sel = scrapy.Selector(text=body_str)

In [186]:
table = sel.css('.sidearm-table')[0]

In [187]:
table

<Selector xpath="descendant-or-self::*[@class and contains(concat(' ', normalize-space(@class), ' '), ' sidearm-table ')]" data='<table class="sidearm-table sidearm-tabl'>

In [188]:
table.css('tr') # Get all the tr elements within the table
table.css('tr')[0] # Get the first tr element within the table using the pythonic index
table.css('tr')[0].css('th') # Get all the th elements from within the first tr
table.css('tr:nth-child(1) th')

[<Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' 

In [189]:
columns = table.css('th').xpath('string()').extract()

In [190]:
columns

['No.', 'Name', 'Pos.', 'Ht.', 'Wt.', 'Yr.', 'Hometown / High School']

In [191]:
rows = table.css('tr')[1:]

In [192]:
by_player_number = []
for r in rows:
    data = {}
    for i, d in enumerate(r.css('td')):
        a = d.css('a')
        if a:
            t = a.xpath('text()').extract()[0]
            data['href'] = a.xpath('@href').extract()[0]
        else:
            t = d.xpath('text()').extract()[0]
        data[columns[i]] = t
    by_player_number.append(data)

In [193]:
by_player_number 
#I also don't understand why its starting with the Hometown - that's weird

[{'Hometown / High School': 'Columbia, S.C. / Hammond School',
  'Ht.': '6-1',
  'Name': 'Seventh Woods',
  'No.': '0',
  'Pos.': 'G',
  'Wt.': '185',
  'Yr.': 'So.',
  'href': '/roster.aspx?rp_id=13521'},
 {'Hometown / High School': 'Greensboro, N.C. / Wesleyan Christian Academy',
  'Ht.': '6-6',
  'Name': 'Theo Pinson',
  'No.': '1',
  'Pos.': 'F/G',
  'Wt.': '220',
  'Yr.': 'Sr.',
  'href': '/roster.aspx?rp_id=13515'},
 {'Hometown / High School': 'Apopka, Fla. / Lake Highland Preparatory',
  'Ht.': '6-0',
  'Name': 'Joel Berry II',
  'No.': '2',
  'Pos.': 'G',
  'Wt.': '195',
  'Yr.': 'Sr.',
  'href': '/roster.aspx?rp_id=13508'},
 {'Hometown / High School': 'Guilderland, N.Y. / Northfield Mount Hermon School (Mass.)',
  'Ht.': '6-3',
  'Name': 'Andrew Platek',
  'No.': '3',
  'Pos.': 'G',
  'Wt.': '195',
  'Yr.': 'Fr.',
  'href': '/roster.aspx?rp_id=13528'},
 {'Hometown / High School': 'Douglasville, Ga. / Douglas County',
  'Ht.': '6-5',
  'Name': 'Brandon Robinson',
  'No.': '4',


In [166]:
def fetch_bio(player):
    player_url = base_url + player['href']
    print('Fetch bio', player_url)
    resp = requests.get(player_url, headers=headers)
    player_txt = resp.content.decode('utf-8')
    sel = scrapy.Selector(text=player_txt)
    player['sel'] = sel
    player['bio'] = sel.css('#sidearm-roster-player-bio').xpath('string()').extract()[0]
    player['img'] = sel.css('.sidearm-roster-player-image img').xpath('@src').extract()[0]

In [167]:
js_obj_rx = re.compile(r'.*?responsive-roster-bio\.ashx.*?(?P<obj>{.*?})')

In [168]:
def fetch_stats(player):
    text = player['sel'].xpath('string()').extract()[0]
    parts = text.split('$.getJSON("/services/')[1:]
    captured = js_obj_rx.findall(''.join(parts))
    clean_objs = []
    for obj_str in captured:
        # We only want the stats object...
        if 'stats' not in obj_str:
            continue

        obj_str = obj_str.replace('{', '').replace('}', '')
        obj_str = obj_str.replace("'", '').replace('"', '')
        obj_pairs = obj_str.split(',')
        obj_pairs = [x.split(":") for x in obj_pairs]
        clean_pairs = []
        for pair in obj_pairs:
            clean_pairs.append(['"{}"'.format(p.strip()) for p in pair])
        colonized = [":".join(p) for p in clean_pairs]
        commas = ','.join(colonized)
        json_str = "{" + commas + "}"
        clean_objs.append(json.loads(json_str))
    
    player['stats_url'] = stats_url = (
        "http://goheels.com/services/responsive-roster-bio.ashx?"
        "type={type}&rp_id={rp_id}&path={path}&year={year}"
        "&player_id={player_id}"
    ).format(**clean_objs[0])
    
    print('Fetch stats', stats_url)
    resp = requests.get(stats_url, headers=headers)
    json_stats = json.loads(resp.content.decode("utf-8"))
    player['raw_stats'] = json_stats

In [194]:
for p in by_player_number:
    fetch_bio(p)
    fetch_stats(p)

Fetch bio http://goheels.com/roster.aspx?rp_id=13521
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=13521&path=mbball&year=2017&player_id=4736
Fetch bio http://goheels.com/roster.aspx?rp_id=13515
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=13515&path=mbball&year=2017&player_id=4636
Fetch bio http://goheels.com/roster.aspx?rp_id=13508
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=13508&path=mbball&year=2017&player_id=4632
Fetch bio http://goheels.com/roster.aspx?rp_id=13528
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=13528&path=mbball&year=2017&player_id=5171
Fetch bio http://goheels.com/roster.aspx?rp_id=13516
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=13516&path=mbball&year=2017&player_id=4733
Fetch bio http://goheels.com/roster.aspx?rp_id=13523
Fetch stats http://goheels.com/services/respons

In [170]:
by_player_number[0]
#how did the bio and stats get appended to the rest of the keys

{'Hometown / High School': 'Columbia, S.C. / Hammond School',
 'Ht.': '6-1',
 'Name': 'Seventh Woods',
 'No.': '0',
 'Pos.': 'G',
 'Wt.': '185',
 'Yr.': 'So.',
 'bio': '\r\n                        Biography\r\n                                                    \r\n                            FRESHMAN SEASON (2016-17)\r\n\r\nTied a school record by appearing in all 40 games • Averaged 7.7 minutes per game in the primary relief role behind Joel Berry II at the point • Had 49 assists, 42 turnovers and 21 steals • Had 25 assists and 13 turnovers in ACC regular-season play • Made two steals five times • Was on the floor for five minutes vs. Louisville as UNC out-scored the Cards, 10-2 • Played his bestgame in conference play at Duke when he scored four points and had a team-high four assists in eight minutes • Played a seven and a half minute stretch in the first half as UNC cut Duke’s lead from three to one • Was the first time he led UNC in assists since Long Beach State on 11/15 • Had t

In [171]:
p = [p for p in by_player_number if p['Name'] == 'Seventh Woods'][0]

In [195]:
txt = p['raw_stats']['current_stats']


In [196]:
sel = scrapy.Selector(text=txt)

In [197]:
sel.css('section')

[<Selector xpath='descendant-or-self::section' data='<section>\r\n        <h5>Game-By-Game Stat'>]

In [210]:
def parse_stats(player):
    stats = {}
    for raw_key, raw_val in player['raw_stats'].items():
        txt = player['raw_stats'][raw_key]
        if not txt:
            print('Skipping {} for {}'.format(raw_key, player['Name']))
            continue
        sel = scrapy.Selector(text=txt)
        # Get all the tables
        for section in sel.css('section'):
            title = section.css('h5').xpath('string()').extract()[0]
            cols = section.css('tr')[0].css('th').xpath('string()').extract()
            print('TITLE OF STATS TABLE', title)
            print('COLUMNS', cols)
            these_stats = []
            print('ROWS', section.css('tr'))
            for r in section.css('tr')[1:]:
                print('row', r.xpath('string()').extract()[0].replace('\r', '').replace('\n', '').strip())
                s = {}
                for i, d in enumerate(r.css('td'), 1):
                    s[cols[i].lower()] = d.xpath('string()').extract()[0]
                opponent = r.css('th').xpath('string()')
                if opponent:
                    opponent = opponent.extract()[0]
                    if opponent.lower() in ('total', 'season'):
                        print('SKIPPING...')
                        continue
                    print('THE OPPONENT', opponent)
                    s['opponent'] = opponent
                these_stats.append(s)
                print('THE STATS ARE', these_stats)
            existing = stats.get(raw_key, {})
            existing[title] = these_stats
            stats[raw_key] = existing
    player['stats'] = stats

In [211]:
p = [p for p in by_player_number if p['Name'] == 'Seventh Woods'][0]
parse_stats(p)
#THIS IS WHERE I GET CONFUSED. WHY IS NORTHERN IOWA REPEATED AGAIN AND AGAIN AND WHY ARE THE REST OF THE GAMES APPEARING DIRECTLY AFTER THE STATS FOR EACH GAME EACH TIME?

TITLE OF STATS TABLE Game-By-Game Statistics
COLUMNS ['Date', 'Opponent', 'GS', 'MIN', 'FGM/A', '%', '3FG/A', '%', 'FTM/A', '%', 'OFF', 'DEF', 'TOT', 'AVG', 'PF', 'AST', 'T/O', 'BLK', 'STL', 'PTS', 'AVG']
ROWS [<Selector xpath='descendant-or-self::tr' data='<tr>\r\n                        <th scope='>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data

IndexError: list index out of range

In [205]:
for p in by_player_number:
    parse_stats(p)

TITLE OF STATS TABLE Game-By-Game Statistics
COLUMNS ['Date', 'Opponent', 'GS', 'MIN', 'FGM/A', '%', '3FG/A', '%', 'FTM/A', '%', 'OFF', 'DEF', 'TOT', 'AVG', 'PF', 'AST', 'T/O', 'BLK', 'STL', 'PTS', 'AVG']
ROWS [<Selector xpath='descendant-or-self::tr' data='<tr>\r\n                        <th scope='>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data

IndexError: list index out of range

In [207]:
[p for p in by_player_number if p['Name'] == 'Seventh Woods'][0]['stats']
#AND WHY IS 'STATS' A KEY ERROR - ITS DEFINED ABOVE

KeyError: 'stats'